# <center> Python Project </center>

## <center> Graphs building and analysis </center>

### Authors: 
#### Martina Pacifici (7005686) and Federica Sauro Graziano (6360850)

![title](https://www.google.com/search?q=logo+unifi&tbm=isch&source=iu&ictx=1&fir=W8Mt4YB506ck_M%252C5XB63tEAzK6DAM%252C_&vet=1&usg=AI4_-kQz_tlNndA-4rsdgXOz5pF2GUJFow&sa=X&ved=2ahUKEwjoi_2UusXqAhXaMMAKHcKwD2UQ9QEwAHoECAoQFA&biw=1366&bih=576#imgrc=aYnSu_DgUmkYuM)

In [2]:
import json
with open("C:\\Users\\feder\\OneDrive\\Documenti\\Fede\\University\\STAT_M1\\AlgoritmiPython\\project\\dpc-covid19-ita-province.json") as f:

         d = json.load(f) 

In [3]:
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import timeit
import random
import math

In [4]:
corona = pd.DataFrame(d)
corona = corona.drop(corona[corona.lat==0.0].index)
corona.reset_index(inplace=True)
df = corona[0:107]

# <center> Graph bulding </center>

After having converted our dataset in a pandas DataFrame, we extract only all the 107 provinces at a particular moment of time (we didn't care about the date). 
In this way we have all the italian provinces with the spatial information of latitude and longitude (in decimal degree) that allow us to to build the graph.

The graph will be a nextwork of provinces called 'P' in which each node corresponds to a city.
Two cities a and b are connected by an edge if they are close in 0.8 decimal degree (that is: if x,y is the position of a, then b is in position z,w with z in [x-d,x+d] and w in [y-d, y+d], with d=0.8).

NetworkX  is a python package useful for the implementation, manipulation, and study of a standard, dynamics and complex graph structure.

First we use networkx to create all the nodes, that are the provinces of a general graph 'G', through the command:

In [5]:
G = nx.Graph()
G.add_nodes_from(corona.sigla_provincia)
G.number_of_nodes()

107

#### First implementation (not efficient)

In [ ]:
%%timeit
for i in range(len(df)):
    for j in range(len(df)):
        if (i != j) & (abs(df.lat[i]-df.lat[j]) <= 0.8) & (abs(df.long[i]-df.long[j]) <= 0.8):
            G.add_edge(df.sigla_provincia[i],df.sigla_provincia[j])

Our first implementation uses two for cicles to find the provinces close (in terms of latitude and longitude) to every node and finally add an edge to the graph 'G' if the proximity requirement is respected.

This algorithm costs **O($n^2$)**, so we tried to find a better solution to add edges.

#### Second implementation (more efficient)

First we need to sort the DataFrame. We created two new sorted DataFrame one respect to longitude and the other respect to latitude. 

To do so, we chose the Mergesort method, which seemed to be the most efficient:

In [7]:
sorted_dfx = df.sort_values(by = 'long', kind = 'mergesort')
sorted_dfx.reset_index(inplace = True)
sorted_dfy = df.sort_values(by = 'lat', kind ='mergesort')
sorted_dfy.reset_index(inplace = True)

This is important in order to use the binarySearch function:

In [8]:
def binarySearch(alist, item, prov, d):
    first = 0
    last = len(alist)-1
    found = False
    while first<=last and not found:
        midpoint = (first + last)//2
        if alist[midpoint]==float(item):
            found = True
            return list(prov[(alist>=float(alist[midpoint]-d)) & (alist<=float(alist[midpoint]+d))])
        else:
            if float(item) < alist[midpoint]:
                last = midpoint-1
            else:
                first = midpoint+1
    return []

Here the function inter_city will help us taking the interesection between the closest cities of a node in respect to both latitudine and longitude.

We used a dictionary to collect the cities to have a more efficient function. In the worst case it takes **O(n)**.

In [9]:
def inter_city(A,B,elem):
    C = {}
    for el in B:
        if el!=elem and el not in C and el in A:
            C[el]=0
    return C

#### Graph P

Finally we can implement our provinces graph:


In [10]:
P = nx.Graph()
P.add_nodes_from(sorted_dfx.sigla_provincia)

In [11]:
%%timeit
for el in sorted_dfx.sigla_provincia:
    lista_legami_x = binarySearch(sorted_dfx.long, sorted_dfx.long[sorted_dfx.sigla_provincia==el], 
                                  sorted_dfx.sigla_provincia, 0.8)
    lista_legami_y = binarySearch(sorted_dfy.lat, sorted_dfy.lat[sorted_dfy.sigla_provincia==el], 
                                  sorted_dfy.sigla_provincia, 0.8)
    lst_x = {j:0 for j in lista_legami_x}
    lst_y = {j:0 for j in lista_legami_y}
    città_vicine = inter_city(lst_x, lst_y, el)
    edge_city = []
    for j in città_vicine:
        edge_city.append((el, j))
    P.add_edges_from(edge_city)

455 ms ± 55 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
P.number_of_edges()

NameError: name 'P' is not defined

In [ ]:
nx.draw(P, with_labels=True, node_color='lightpink')

We can observe that this node is made by four different components:

The province of Trapani is an isolated node;
All the provinces of Sardinia make a single component;
All the provinces of Calabria and Sicily regions (except from Trapani) make a single component;
The rest of Italy's provinces are connected by edges. 

#### Graph R

We now use our second implemention to build a larger graph R. 

First we generate 2000 pairs of double (x,y) with x in [30,50) and y in [10,20], which will correspond to fictitious provinces with their longitude (x) and latitude (y), and insert them in a DataFrame:

In [ ]:
latit = [] 
longit = []  
nodes = [] 
for i in range(2000):
    latit.append(random.uniform(30,50))
    longit.append(random.uniform(10,20))
    nodes.append(str(i))
    
dataRandom = {'province': nodes,
              'longitudine' : longit,
              'latitudine' : latit}    

dataR = pd.DataFrame(dataRandom, columns = ['province', 'longitudine', 'latitudine'])

Again, after building a new graph R, we create new DataFrames sorting dataR once for x and once for y: 

In [ ]:
R = nx.Graph()
R.add_nodes_from(dataR.province)
R.number_of_nodes()

sort_dataRx= dataR.sort_values(by = 'longitudine', kind = 'mergesort')
sort_dataRx.reset_index(inplace = True)
sort_dataRy = dataR.sort_values(by = 'latitudine', kind ='mergesort')
sort_dataRy.reset_index(inplace = True)

Now we add edges to the graph R for the provinces that has both latitude and longitude within 0.08 degree from the ones of the node:

In [ ]:
%%timeit
for el in sort_dataRx.province:
    legami_x = binarySearch(sort_dataRx.longitudine, sort_dataRx.longitudine[sort_dataRx.province==el], sort_dataRx.province, 0.08)
    legami_y = binarySearch(sort_dataRy.latitudine, sort_dataRy.latitudine[sort_dataRy.province==el], sort_dataRy.province, 0.08)
    lst_x = {j:0 for j in lista_legami_x}
    lst_y = {j:0 for j in lista_legami_y}
    città_vicine = inter_city(lst_x, lst_y, el)
    edge_city = []
    for j in città_vicine:
        edge_city.append((el, j))
    R.add_edges_from(edge_city)

In [ ]:
R.number_of_edges()

In [ ]:
Here we show how the graph R appears: 

In [ ]:
nx.draw(R, node_color='red')

#### Weighted Graph

Now we weight both the graph P and R for the distance. 

In [ ]:
#%%timeit
for el in sorted_dfx.sigla_provincia:
    lista_legami_x = binarySearch(sorted_dfx.long, sorted_dfx.long[sorted_dfx.sigla_provincia==el], sorted_dfx.sigla_provincia, 0.8)
    lista_legami_y = binarySearch(sorted_dfy.lat, sorted_dfy.lat[sorted_dfy.sigla_provincia==el], sorted_dfy.sigla_provincia, 0.8)
    lst_x = {j:0 for j in lista_legami_x}
    lst_y = {j:0 for j in lista_legami_y}
    città_vicine = inter_city(lst_x, lst_y, el)
    edge_city = []
    for j in città_vicine:
        distanza_x = (float(sorted_dfx.long[sorted_dfx.sigla_provincia==el])-float(sorted_dfx.long[sorted_dfx.sigla_provincia==j]))**2
        distanza_y = (float(sorted_dfy.lat[sorted_dfy.sigla_provincia==el])-float(sorted_dfy.lat[sorted_dfy.sigla_provincia==j]))**2
        distanza = math.sqrt(distanza_x+distanza_y)
        edge_city.append((el, j, distanza))
    P.add_weighted_edges_from(edge_city)
               

P.number_of_edges()
nx.draw(P)

In [ ]:
for el in sort_dataRx.province:
    legami_x = binarySearch(sort_dataRx.longitudine, sort_dataRx.longitudine[sort_dataRx.province==el], sort_dataRx.province, 0.08)
    legami_y = binarySearch(sort_dataRy.latitudine, sort_dataRy.latitudine[sort_dataRy.province==el], sort_dataRy.province, 0.08)
    lst_x = {j:0 for j in lista_legami_x}
    lst_y = {j:0 for j in lista_legami_y}
    città_vicine = inter_city(lst_x, lst_y, el)
    edge_city = []
    for j in città_vicine:
        distanza_x = ((sort_dataRx.longitudine[sort_dataRx.province==el])-(sort_dataRx.longitudine[sort_dataRx.province==j]))**2
        distanza_y = ((sort_dataRy.latitudine[sort_dataRy.province==el])-(sort_dataRy.latitudine[sort_dataRy.province==j]))**2
        distanza = math.sqrt(distanza_x+distanza_y)
        edge_city.append((el, j, distanza))
    R.add_weighted_edges_from(edge_city)
 
R.number_of_edges()
nx.draw(R)

#### Bellman Ford

We decided to use Bellman Ford algorithm in order to get the shortest path from a single source vertex to all the others part of the same weighted graph. 

Bellman-Ford algorithm proceeds by relaxation, in which approximations to the correct distance are replaced by better ones until they eventually reach the solution.

It is comparable with Dijkstra's algorithm for how they work. In both algorithms, the approximate distance to each vertex is always an overestimate of the true distance, and is replaced by the minimum of its old value and the length of a newly found path.

Even if Bellman-Ford algorithm is slower than Dijkstra's, it is able to deal with graphs in which some of the edge weights are negative numbers. For sure this is not our case because the weights of our graphs are geographical distances and they can never be negative.

Here we analyze the three steps of the algorithm. 

We start it inizializing the distance to the source vertex to 0 and all the other nodes to infinity:

In [ ]:
def bell_ford(graph, source):
    
    #inizialization
    dist = {}
    pred = {}
    
    for v in graph.node():
        dist[v] = math.inf
        pred[v] = None
    
    dist[source] = 0

After inizializing we proceed with the relaxation: for all edges, if the distance to the destination can be shortened by taking the edge, the distance is updated to the new lower value.

At each iteration i that the edges are scanned, the algorithm finds all shortest paths of at most length i edges (and possibly some paths longer than i edges). 

If the graph is connected we get at most |V|-1 edges in our path. 

In [ ]:
 #relaxation
    for i in range(len(graph)-1): #Run this until is converges
        for u in graph:
            for v in graph[u]: #For each neighbour of u
                if dist[v] > dist[u] + graph[u][v]['weight']:
                    # Record this lower distance
                    dist[v]  = dist[u] + graph[u][v]['weight']
                    pred[v] = u
                if dist[u] > dist[v] + graph[u][v]['weight']:
                    # Record this lower distance
                    dist[u]  = dist[v] + graph[u][v]['weight']
                    pred[u] = v
    

The last step of the algorithm is needed to detect the presence of negative-weight cycle. In fact, without this part, if the algorithm finds a nevative-weight cycle, it would enter in an Infinite Loop. 

Thanks to this third step if it happens to find one negative-weight cycle the algorithm will return an AssertionError, while if there aren't it will go on returning the shortest path from the source vertex to all the others.

Finally the algorithm returns the shortest path in terms of distance and the predecessor to the final node. 

In [ ]:
#check for negative-weight cycles
    for u in graph:
        for v in graph[u]:
            assert dist[v] <= dist[u] + graph[u][v]['weight']
    return dist, pred

Here we show some results for both the weighted graph P and R:

In [15]:
bell_ford(P, 'OR')[0]

NameError: name 'P' is not defined

In [ ]:
bell_ford(P, 'TP')[0]

In [ ]:
bell_ford(P, 'FI')[0]

In [ ]:
bell_ford(P, 'KR')[0]

In [ ]:
bell_ford(R, ('2'))[0]

Both graphs P and R are not connected, so the path from a source vertex of a specific compontent to other nodes that aren't part of that component is set to Infinity. 

In fact we can see that the province of Trapani, which builds itself a component, has all the paths set to infinity except for the one with itself, which is 0. 

#### Closeness Centrality

In a connected graph, the closeness centrality of a node is a measure of centrality in a network, calculated as the reciprocal of the sum of the length of the shortest paths between the node and all other nodes in the graph.

The closeness centrality of a node v in a graph V is defined as:

$$c(v) =\frac {n−1}{f(v)}$$

where n are the number of nodes and f(v) is the farness of a node v equal to $\Sigma_{w \in V}d(v,w)$, that is the sum of the distances between the two vertices v and w.

Since we don't have a connected graph we use its variance in which, instead of the reciprocal of the sum, we sum the reciprocal of the distances (infact, when a distance is infinite we'll have to sum $\frac{1}{\infty}=0$).

We first define a closeness centrality algorithm based on the shortest paths computed with **Bellman-Ford**:

In [ ]:
def closeness(graph, u):
    dizy_dist = bell_ford(graph, u)[0]
    close = 0.0
    for key in dizy_dist:
        if dizy_dist[key] != 0 and dizy_dist[key] != math.inf:
            close = close + 1/dizy_dist[key]
    return close

Here some results:

In [ ]:
closeness(P,'TP')

In [ ]:
closeness(P,'FI') 

For a better interpretation we now use an algorithm which returns the **normalized** closeness index based on **Bellman-Ford** distance: 

In [ ]:
def closeness_norm_dist(graph, u):
    dizy_dist = bell_ford(graph, u)[0]
    total = 0
    n_short_path = 0
    cc= 0.0
    for key in dizy_dist:
        if dizy_dist[key] != 0 and dizy_dist[key] != math.inf:
            total = total + dizy_dist[key]
            n_short_path+=1
    if total > 0.0 and len(graph)>1:
        s = (n_short_path)/(len(graph)-1)
        cc = ((n_short_path)/total)*s
    return cc

We now show the closeness centrality normalized index again for Trapani and Florencia provinces: 

In [ ]:
closeness_norm_dist(P,'TP')

In [ ]:
closeness_norm_dist(P,'FI')

Furthermore we implemented the same **normalized** closeness algorithm but using the **shortest path length** of *NetworkX* (without specificating weights) instead of Bellman-Ford:

In [ ]:
def closeness_norm_short(graph, u):
    dizy_dist = nx.shortest_path_length(graph, u)
    total = 0
    n_short_path = 0
    cc = 0.0
    for key in dizy_dist:
        if dizy_dist[key] != 0 and dizy_dist[key] != math.inf:
            total = total + dizy_dist[key]
            n_short_path+=1
    if total > 0.0 and len(graph)>1:
        s = (n_short_path)/(len(graph)-1)
        cc = ((n_short_path)/total)*s
    return cc    

In [ ]:
closeness_norm_short(P,'TP')

In [ ]:
closeness_norm_short(P,'FI')

In [ ]:
closeness_norm_short(R,'2')

Finally through the following algorithm we'll get the **most central node** of a graph based on our second closeness algorithm (which uses Bellman-Ford distance):

In [ ]:
def massimaClosy(graph):
    maximum = 0.0
    city = ''
    for v in graph.nodes():
        closy = closeness_norm_dist(graph,v)
        if closy >= maximum:
            maximum = closy
            city = v
    return city, maximum

massimaClosy(P)

Or, alternatively, in a more efficient way: 

In [ ]:
def get_key(val, my_dict): 
    for key, value in my_dict.items(): 
         if val == value: 
             return key 
    return "key doesn't exist"

dist_max = max(list(nx.closeness_centrality(P,distance = 'weight').values()))
get_key(dist_max,nx.closeness_centrality(P,distance = 'weight'))

# Thank you for your attention!